In [1]:
import pandas as pd
from sklearn.preprocessing import label_binarize
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings("ignore")

In [2]:
R_train = pd.read_csv('/Users/jiwonmoon/Desktop/CIS 508/Data Mining I/Assignments/Individual Assignment 3/data/RevisedHomesiteTrain1.csv')
display(R_train)

train = pd.DataFrame(R_train)

,CoverageField11A,CoverageField11B,CoverageField1A,CoverageField1B,CoverageField2A,CoverageField2B,CoverageField3A,CoverageField3B,CoverageField4A,CoverageField4B,...,PropertyField38_N,PropertyField38_Y,GeographicField63_,GeographicField63_N,GeographicField63_Y,GeographicField64_CA,GeographicField64_IL,GeographicField64_NJ,GeographicField64_TX,QuoteConversion_Flag
0,2,1,17,23,17,23,15,22,16,22,...,1,0,0,1,0,1,0,0,0,0
1,5,9,6,8,6,8,5,7,5,8,...,1,0,0,1,0,0,0,1,0,0
2,4,6,7,12,7,12,6,10,7,11,...,1,0,0,1,0,0,0,1,0,0
3,15,23,3,2,3,2,2,2,3,2,...,1,0,0,1,0,0,0,0,1,0
4,4,6,8,13,8,13,7,11,7,13,...,1,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64995,3,5,9,15,9,15,8,13,8,14,...,1,0,0,1,0,1,0,0,0,0
64996,5,9,14,21,14,21,13,20,13,21,...,1,0,0,1,0,0,0,0,1,0
64997,7,14,3,2,3,3,2,2,3,2,...,1,0,0,1,0,0,0,1,0,0
64998,5,9,15,22,15,22,13,21,14,21,...,1,0,0,1,0,0,0,1,0,0


In [3]:
R_test = pd.read_csv('/Users/jiwonmoon/Desktop/CIS 508/Data Mining I/Assignments/Individual Assignment 3/data/RevisedHomesiteTest1.csv')
display(R_test)

test = pd.DataFrame(R_test)

,CoverageField11A,CoverageField11B,CoverageField1A,CoverageField1B,CoverageField2A,CoverageField2B,CoverageField3A,CoverageField3B,CoverageField4A,CoverageField4B,...,PropertyField38_N,PropertyField38_Y,GeographicField63_,GeographicField63_N,GeographicField63_Y,GeographicField64_CA,GeographicField64_IL,GeographicField64_NJ,GeographicField64_TX,GeographicField64
0,13,22,4,4,4,4,3,3,3,4,...,1,0,0,0,1,0,0,0,0,IL
1,4,5,8,14,8,14,7,12,8,13,...,1,0,0,1,0,0,0,0,0,NJ
2,3,3,11,18,11,18,10,16,10,18,...,1,0,0,1,0,0,0,0,0,NJ
3,5,9,14,22,15,22,13,20,22,25,...,1,0,0,1,0,0,0,0,0,TX
4,12,21,4,5,4,5,4,4,4,5,...,1,0,0,1,0,0,0,0,0,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173831,14,23,13,21,13,21,18,23,13,20,...,0,1,0,1,0,0,0,0,0,NJ
173832,7,13,10,17,10,17,9,15,9,16,...,1,0,0,1,0,0,0,0,0,CA
173833,12,22,4,4,4,5,4,4,4,4,...,1,0,0,1,0,0,0,0,0,CA
173834,6,11,4,5,4,5,4,4,4,4,...,1,0,0,1,0,0,0,0,0,TX


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65000 entries, 0 to 64999
Columns: 596 entries, CoverageField11A to QuoteConversion_Flag
dtypes: float64(5), int64(591)
memory usage: 295.6 MB


In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173836 entries, 0 to 173835
Columns: 596 entries, CoverageField11A to GeographicField64
dtypes: float64(5), int64(590), object(1)
memory usage: 790.5+ MB


# Preprocessing

In [6]:
col_test = test.columns
col_train = train.columns

s = set(col_train)
missing = [x for x in col_test if x not in s]
missing

['GeographicField64']

In [7]:
col_test = test.columns
col_train = train.columns

s = set(col_test)
missing = [x for x in col_train if x not in s]
missing

['QuoteConversion_Flag']

In [8]:
test = test.drop(['GeographicField64'], axis=1)
test

,CoverageField11A,CoverageField11B,CoverageField1A,CoverageField1B,CoverageField2A,CoverageField2B,CoverageField3A,CoverageField3B,CoverageField4A,CoverageField4B,...,PropertyField37_Y,PropertyField38_N,PropertyField38_Y,GeographicField63_,GeographicField63_N,GeographicField63_Y,GeographicField64_CA,GeographicField64_IL,GeographicField64_NJ,GeographicField64_TX
0,13,22,4,4,4,4,3,3,3,4,...,1,1,0,0,0,1,0,0,0,0
1,4,5,8,14,8,14,7,12,8,13,...,0,1,0,0,1,0,0,0,0,0
2,3,3,11,18,11,18,10,16,10,18,...,0,1,0,0,1,0,0,0,0,0
3,5,9,14,22,15,22,13,20,22,25,...,0,1,0,0,1,0,0,0,0,0
4,12,21,4,5,4,5,4,4,4,5,...,0,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173831,14,23,13,21,13,21,18,23,13,20,...,0,0,1,0,1,0,0,0,0,0
173832,7,13,10,17,10,17,9,15,9,16,...,0,1,0,0,1,0,0,0,0,0
173833,12,22,4,4,4,5,4,4,4,4,...,1,1,0,0,1,0,0,0,0,0
173834,6,11,4,5,4,5,4,4,4,4,...,1,1,0,0,1,0,0,0,0,0


# Modeling (DTC, RFC,LinearSVM, MLPClassifier, KNN)

In [9]:
#Separate Train data and test data

X = train.drop("QuoteConversion_Flag", axis=1)
y = train["QuoteConversion_Flag"]

X_train, X_test, y_train, y_test = train_test_split (X, y, test_size=0.2,random_state=0)

## 1. DTC

In [10]:
dt = DecisionTreeClassifier()

dt.fit(X_train, y_train)
dt_predict = dt.predict(X_test)

print("accuracy Score (training) for DecisionTreeClassifier:{0:6f}".format(dt.score(X_test,y_test)))
print("Confusion Matrix for DecisionTreeClassifier:")
print(confusion_matrix(y_test,dt_predict))
print('Classification Report :')
print(classification_report(y_test,dt_predict))

accuracy Score (training) for DecisionTreeClassifier:0.881385
Confusion Matrix for DecisionTreeClassifier:
[[9746  761]
 [ 781 1712]]
Classification Report :
              precision    recall  f1-score   support

           0       0.93      0.93      0.93     10507
           1       0.69      0.69      0.69      2493

    accuracy                           0.88     13000
   macro avg       0.81      0.81      0.81     13000
weighted avg       0.88      0.88      0.88     13000



## 2. RFC

In [11]:
rfc = RandomForestClassifier()

rfc.fit(X_train, y_train)
rfc_predict = rfc.predict(X_test)

print("accuracy Score (training) for RandomForest:{0:6f}".format(rfc.score(X_test,y_test)))
print("Confusion Matrix for RandomForest:")
print(confusion_matrix(y_test,rfc_predict))
print('Classification Report for RandomForest:')
print(classification_report(y_test,rfc_predict))

accuracy Score (training) for RandomForest:0.902538
Confusion Matrix for RandomForest:
[[10338   169]
 [ 1098  1395]]
Classification Report for RandomForest:
              precision    recall  f1-score   support

           0       0.90      0.98      0.94     10507
           1       0.89      0.56      0.69      2493

    accuracy                           0.90     13000
   macro avg       0.90      0.77      0.81     13000
weighted avg       0.90      0.90      0.89     13000



## 3. LinearSVM

In [12]:
from sklearn.svm import LinearSVC 

linsvm = LinearSVC(max_iter=300) 

linsvm.fit(X_train, y_train) 
linsvm_predict = linsvm.predict(X_test) 

print("accuracy Score (training) for LinearSVMClassifier:{0:6f}".format(linsvm.score(X_test,y_test))) 
print("Confusion Matrix for LinearSVMClassifier:") 
print(confusion_matrix(y_test,linsvm_predict))
print('Classification Report for LinearSVMClassifier:')
print(classification_report(y_test,linsvm_predict))

accuracy Score (training) for LinearSVMClassifier:0.807923
Confusion Matrix for LinearSVMClassifier:
[[10484    23]
 [ 2474    19]]
Classification Report for LinearSVMClassifier:
              precision    recall  f1-score   support

           0       0.81      1.00      0.89     10507
           1       0.45      0.01      0.01      2493

    accuracy                           0.81     13000
   macro avg       0.63      0.50      0.45     13000
weighted avg       0.74      0.81      0.73     13000



## 4. MLPClassifier

In [13]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(max_iter=100)

mlp.fit(X_train, y_train)
mlp_predict = mlp.predict(X_test)

print("accuracy Score (training) for MultiLayerPerceptron:{0:6f}".format(mlp.score(X_test,y_test)))
print("Confusion Matrix for MultiLayerPerceptron:")
print(confusion_matrix(y_test,mlp_predict))
print('Classification Report MultiLayerPerceptron:')
print(classification_report(y_test,mlp_predict))

accuracy Score (training) for MultiLayerPerceptron:0.843462
Confusion Matrix for MultiLayerPerceptron:
[[9953  554]
 [1481 1012]]
Classification Report MultiLayerPerceptron:
              precision    recall  f1-score   support

           0       0.87      0.95      0.91     10507
           1       0.65      0.41      0.50      2493

    accuracy                           0.84     13000
   macro avg       0.76      0.68      0.70     13000
weighted avg       0.83      0.84      0.83     13000



## 5. KNN

In [14]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)

knn.fit(X_train, y_train)
knn_predict = knn.predict(X_test)

print("accuracy Score (training) for KNeighborsClassifier:{0:6f}".format(knn.score(X_test,y_test)))
print("Confusion Matrix for KNeighborsClassifier:")
print(confusion_matrix(y_test,knn_predict))
print('Classification Report KNeighborsClassifier:')
print(classification_report(y_test,knn_predict))

accuracy Score (training) for KNeighborsClassifier:0.754615
Confusion Matrix for KNeighborsClassifier:
[[9527  980]
 [2210  283]]
Classification Report KNeighborsClassifier:
              precision    recall  f1-score   support

           0       0.81      0.91      0.86     10507
           1       0.22      0.11      0.15      2493

    accuracy                           0.75     13000
   macro avg       0.52      0.51      0.50     13000
weighted avg       0.70      0.75      0.72     13000



# SMOTE With Different Ratio and Classifier Scores Respectively

In [15]:
conda install -c conda-forge imbalanced-learn

Solving environment: done

# All requested packages already installed.

Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [16]:
from imblearn.over_sampling import SMOTE 
from collections import Counter

## 1. SMOTE = 0.5

In [17]:
# SMOTE = 0.5

print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(sampling_strategy= 0.5)
X_res1, y_res1 = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res1))

___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 42231, 1: 9769})
Resampled dataset shape Counter({0: 42231, 1: 21115})


In [18]:
# DTC

dt1 = DecisionTreeClassifier()

dt1.fit(X_res1, y_res1)
dt1_predict = dt1.predict(X_test)

print("accuracy Score (training) for DecisionTreeClassifier:{0:6f}".format(dt1.score(X_test,y_test)))
print("Confusion Matrix for DecisionTreeClassifier:")
print(confusion_matrix(y_test,dt1_predict))
print('Classification Report :')
print(classification_report(y_test,dt1_predict))

accuracy Score (training) for DecisionTreeClassifier:0.877077
Confusion Matrix for DecisionTreeClassifier:
[[9686  821]
 [ 777 1716]]
Classification Report :
              precision    recall  f1-score   support

           0       0.93      0.92      0.92     10507
           1       0.68      0.69      0.68      2493

    accuracy                           0.88     13000
   macro avg       0.80      0.81      0.80     13000
weighted avg       0.88      0.88      0.88     13000



In [19]:
# RFC

rfc1 = RandomForestClassifier()

rfc1.fit(X_res1, y_res1)
rfc1_predict = rfc1.predict(X_test)

print("accuracy Score (training) for RandomForest:{0:6f}".format(rfc1.score(X_test,y_test)))
print("Confusion Matrix for RandomForest:")
print(confusion_matrix(y_test,rfc1_predict))
print('Classification Report for RandomForest:')
print(classification_report(y_test,rfc1_predict))

accuracy Score (training) for RandomForest:0.898769
Confusion Matrix for RandomForest:
[[10197   310]
 [ 1006  1487]]
Classification Report for RandomForest:
              precision    recall  f1-score   support

           0       0.91      0.97      0.94     10507
           1       0.83      0.60      0.69      2493

    accuracy                           0.90     13000
   macro avg       0.87      0.78      0.82     13000
weighted avg       0.89      0.90      0.89     13000



In [20]:
# LinearSVM

linsvm1 = LinearSVC(max_iter=300) 

linsvm1.fit(X_res1, y_res1) 
linsvm1_predict = linsvm1.predict(X_test) 

print("accuracy Score (training) for LinearSVMClassifier:{0:6f}".format(linsvm1.score(X_test,y_test))) 
print("Confusion Matrix for LinearSVMClassifier:") 
print(confusion_matrix(y_test,linsvm1_predict))
print('Classification Report for LinearSVMClassifier:')
print(classification_report(y_test,linsvm1_predict))

# -> f1-score for 1 is too low

accuracy Score (training) for LinearSVMClassifier:0.806000
Confusion Matrix for LinearSVMClassifier:
[[10404   103]
 [ 2419    74]]
Classification Report for LinearSVMClassifier:
              precision    recall  f1-score   support

           0       0.81      0.99      0.89     10507
           1       0.42      0.03      0.06      2493

    accuracy                           0.81     13000
   macro avg       0.61      0.51      0.47     13000
weighted avg       0.74      0.81      0.73     13000



In [21]:
# MLPClassifier

mlp1 = MLPClassifier(max_iter=100)

mlp1.fit(X_res1, y_res1)
mlp1_predict = mlp1.predict(X_test)

print("accuracy Score (training) for MultiLayerPerceptron:{0:6f}".format(mlp1.score(X_test,y_test)))
print("Confusion Matrix for MultiLayerPerceptron:")
print(confusion_matrix(y_test,mlp1_predict))
print('Classification Report MultiLayerPerceptron:')
print(classification_report(y_test,mlp1_predict))

# -> f1-score for 1 is too low

accuracy Score (training) for MultiLayerPerceptron:0.854000
Confusion Matrix for MultiLayerPerceptron:
[[10268   239]
 [ 1659   834]]
Classification Report MultiLayerPerceptron:
              precision    recall  f1-score   support

           0       0.86      0.98      0.92     10507
           1       0.78      0.33      0.47      2493

    accuracy                           0.85     13000
   macro avg       0.82      0.66      0.69     13000
weighted avg       0.84      0.85      0.83     13000



In [22]:
# KNN

knn1 = KNeighborsClassifier(n_neighbors=3)

knn1.fit(X_res1, y_res1)
knn1_predict = knn1.predict(X_test)
print("accuracy Score (training) for KNeighborsClassifier:{0:6f}".format(knn1.score(X_test,y_test)))
print("Confusion Matrix for KNeighborsClassifier:")
print(confusion_matrix(y_test,knn1_predict))
print('Classification Report KNeighborsClassifier:')
print(classification_report(y_test,knn1_predict))

# -> f1-score is too low

accuracy Score (training) for KNeighborsClassifier:0.659692
Confusion Matrix for KNeighborsClassifier:
[[7873 2634]
 [1790  703]]
Classification Report KNeighborsClassifier:
              precision    recall  f1-score   support

           0       0.81      0.75      0.78     10507
           1       0.21      0.28      0.24      2493

    accuracy                           0.66     13000
   macro avg       0.51      0.52      0.51     13000
weighted avg       0.70      0.66      0.68     13000



## 2. SMOTE = 0.7

In [23]:
# SMOTE = 0.7

print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(sampling_strategy= 0.7)
X_res2, y_res2 = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res2))

___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 42231, 1: 9769})
Resampled dataset shape Counter({0: 42231, 1: 29561})


In [24]:
# DTC

dt2 = DecisionTreeClassifier()

dt2.fit(X_res2, y_res2)
dt2_predict = dt2.predict(X_test)

print("accuracy Score (training) for DecisionTreeClassifier:{0:6f}".format(dt2.score(X_test,y_test)))
print("Confusion Matrix for DecisionTreeClassifier:")
print(confusion_matrix(y_test,dt2_predict))
print('Classification Report :')
print(classification_report(y_test,dt2_predict))

# -> working well, f1-score decreased

accuracy Score (training) for DecisionTreeClassifier:0.871923
Confusion Matrix for DecisionTreeClassifier:
[[9630  877]
 [ 788 1705]]
Classification Report :
              precision    recall  f1-score   support

           0       0.92      0.92      0.92     10507
           1       0.66      0.68      0.67      2493

    accuracy                           0.87     13000
   macro avg       0.79      0.80      0.80     13000
weighted avg       0.87      0.87      0.87     13000



In [25]:
# RFC

rfc2 = RandomForestClassifier()

rfc2.fit(X_res2, y_res2)
rfc2_predict = rfc2.predict(X_test)

print("accuracy Score (training) for RandomForest:{0:6f}".format(rfc2.score(X_test,y_test)))
print("Confusion Matrix for RandomForest:")
print(confusion_matrix(y_test,rfc2_predict))
print('Classification Report for RandomForest:')
print(classification_report(y_test,rfc2_predict))

# -> working well

accuracy Score (training) for RandomForest:0.898000
Confusion Matrix for RandomForest:
[[10152   355]
 [  971  1522]]
Classification Report for RandomForest:
              precision    recall  f1-score   support

           0       0.91      0.97      0.94     10507
           1       0.81      0.61      0.70      2493

    accuracy                           0.90     13000
   macro avg       0.86      0.79      0.82     13000
weighted avg       0.89      0.90      0.89     13000



In [26]:
# LinearSVM

linsvm2 = LinearSVC(max_iter=300) 

linsvm2.fit(X_res2, y_res2) 
linsvm2_predict = linsvm2.predict(X_test) 

print("accuracy Score (training) for LinearSVMClassifier:{0:6f}".format(linsvm2.score(X_test,y_test))) 
print("Confusion Matrix for LinearSVMClassifier:") 
print(confusion_matrix(y_test,linsvm2_predict))
print('Classification Report for LinearSVMClassifier:')
print(classification_report(y_test,linsvm2_predict))

# -> f1-score is too low

accuracy Score (training) for LinearSVMClassifier:0.682923
Confusion Matrix for LinearSVMClassifier:
[[7861 2646]
 [1476 1017]]
Classification Report for LinearSVMClassifier:
              precision    recall  f1-score   support

           0       0.84      0.75      0.79     10507
           1       0.28      0.41      0.33      2493

    accuracy                           0.68     13000
   macro avg       0.56      0.58      0.56     13000
weighted avg       0.73      0.68      0.70     13000



In [27]:
# MLPClassifier

mlp2 = MLPClassifier(max_iter=100)

mlp2.fit(X_res2, y_res2)
mlp2_predict = mlp2.predict(X_test)

print("accuracy Score (training) for MultiLayerPerceptron:{0:6f}".format(mlp2.score(X_test,y_test)))
print("Confusion Matrix for MultiLayerPerceptron:")
print(confusion_matrix(y_test,mlp2_predict))
print('Classification Report MultiLayerPerceptron:')
print(classification_report(y_test,mlp2_predict))

# -> f1-score is too low

accuracy Score (training) for MultiLayerPerceptron:0.859769
Confusion Matrix for MultiLayerPerceptron:
[[9960  547]
 [1276 1217]]
Classification Report MultiLayerPerceptron:
              precision    recall  f1-score   support

           0       0.89      0.95      0.92     10507
           1       0.69      0.49      0.57      2493

    accuracy                           0.86     13000
   macro avg       0.79      0.72      0.74     13000
weighted avg       0.85      0.86      0.85     13000



In [28]:
# KNN

knn2 = KNeighborsClassifier(n_neighbors=3)

knn2.fit(X_res2, y_res2)
knn2_predict = knn2.predict(X_test)
print("accuracy Score (training) for KNeighborsClassifier:{0:6f}".format(knn2.score(X_test,y_test)))
print("Confusion Matrix for KNeighborsClassifier:")
print(confusion_matrix(y_test,knn2_predict))
print('Classification Report KNeighborsClassifier:')
print(classification_report(y_test,knn2_predict))

# -> f1-score is low

accuracy Score (training) for KNeighborsClassifier:0.627385
Confusion Matrix for KNeighborsClassifier:
[[7308 3199]
 [1645  848]]
Classification Report KNeighborsClassifier:
              precision    recall  f1-score   support

           0       0.82      0.70      0.75     10507
           1       0.21      0.34      0.26      2493

    accuracy                           0.63     13000
   macro avg       0.51      0.52      0.51     13000
weighted avg       0.70      0.63      0.66     13000



## 3. SMOTE = 0.9

In [29]:
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(sampling_strategy= 0.9)
X_res3, y_res3 = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res3))

___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 42231, 1: 9769})
Resampled dataset shape Counter({0: 42231, 1: 38007})


In [30]:
# DTC

dt3 = DecisionTreeClassifier()

dt3.fit(X_res3, y_res3)
dt3_predict = dt3.predict(X_test)

print("accuracy Score (training) for DecisionTreeClassifier:{0:6f}".format(dt3.score(X_test,y_test)))
print("Confusion Matrix for DecisionTreeClassifier:")
print(confusion_matrix(y_test,dt3_predict))
print('Classification Report :')
print(classification_report(y_test,dt3_predict))

# -> working well, similar with 0.7

accuracy Score (training) for DecisionTreeClassifier:0.871077
Confusion Matrix for DecisionTreeClassifier:
[[9612  895]
 [ 781 1712]]
Classification Report :
              precision    recall  f1-score   support

           0       0.92      0.91      0.92     10507
           1       0.66      0.69      0.67      2493

    accuracy                           0.87     13000
   macro avg       0.79      0.80      0.80     13000
weighted avg       0.87      0.87      0.87     13000



In [31]:
# RFC

rfc3 = RandomForestClassifier()

rfc3.fit(X_res3, y_res3)
rfc3_predict=rfc3.predict(X_test)

print("accuracy Score (training) for RandomForest:{0:6f}".format(rfc3.score(X_test,y_test)))
print("Confusion Matrix for RandomForest:")
print(confusion_matrix(y_test,rfc3_predict))
print('Classification Report for RandomForest:')
print(classification_report(y_test,rfc3_predict))

# -> working well, f1-score increased

accuracy Score (training) for RandomForest:0.899308
Confusion Matrix for RandomForest:
[[10138   369]
 [  940  1553]]
Classification Report for RandomForest:
              precision    recall  f1-score   support

           0       0.92      0.96      0.94     10507
           1       0.81      0.62      0.70      2493

    accuracy                           0.90     13000
   macro avg       0.86      0.79      0.82     13000
weighted avg       0.89      0.90      0.89     13000



In [32]:
# LinearSVM

linsvm3 = LinearSVC(max_iter=300) 

linsvm3.fit(X_res3, y_res3) 
linsvm3_predict = linsvm3.predict(X_test) 

print("accuracy Score (training) for LinearSVMClassifier:{0:6f}".format(linsvm3.score(X_test,y_test))) 
print("Confusion Matrix for LinearSVMClassifier:") 
print(confusion_matrix(y_test,linsvm3_predict))
print('Classification Report for LinearSVMClassifier:')
print(classification_report(y_test,linsvm3_predict))

# -> f1-score is too low, not appropriate

accuracy Score (training) for LinearSVMClassifier:0.574077
Confusion Matrix for LinearSVMClassifier:
[[6036 4471]
 [1066 1427]]
Classification Report for LinearSVMClassifier:
              precision    recall  f1-score   support

           0       0.85      0.57      0.69     10507
           1       0.24      0.57      0.34      2493

    accuracy                           0.57     13000
   macro avg       0.55      0.57      0.51     13000
weighted avg       0.73      0.57      0.62     13000



In [33]:
# MLPClassifier

mlp3 = MLPClassifier(max_iter=100)

mlp3.fit(X_res3, y_res3)
mlp3_predict = mlp3.predict(X_test)

print("accuracy Score (training) for MultiLayerPerceptron:{0:6f}".format(mlp3.score(X_test,y_test)))
print("Confusion Matrix for MultiLayerPerceptron:")
print(confusion_matrix(y_test,mlp3_predict))
print('Classification Report MultiLayerPerceptron:')
print(classification_report(y_test,mlp3_predict))

accuracy Score (training) for MultiLayerPerceptron:0.550538
Confusion Matrix for MultiLayerPerceptron:
[[4823 5684]
 [ 159 2334]]
Classification Report MultiLayerPerceptron:
              precision    recall  f1-score   support

           0       0.97      0.46      0.62     10507
           1       0.29      0.94      0.44      2493

    accuracy                           0.55     13000
   macro avg       0.63      0.70      0.53     13000
weighted avg       0.84      0.55      0.59     13000



In [34]:
# KNN

knn3 = KNeighborsClassifier(n_neighbors=3)

knn3.fit(X_res3, y_res3)
knn3_predict = knn3.predict(X_test)
print("accuracy Score (training) for KNeighborsClassifier:{0:6f}".format(knn3.score(X_test,y_test)))
print("Confusion Matrix for KNeighborsClassifier:")
print(confusion_matrix(y_test,knn3_predict))
print('Classification Report KNeighborsClassifier:')
print(classification_report(y_test,knn3_predict))

# -> f1-score is low

accuracy Score (training) for KNeighborsClassifier:0.603077
Confusion Matrix for KNeighborsClassifier:
[[6893 3614]
 [1546  947]]
Classification Report KNeighborsClassifier:
              precision    recall  f1-score   support

           0       0.82      0.66      0.73     10507
           1       0.21      0.38      0.27      2493

    accuracy                           0.60     13000
   macro avg       0.51      0.52      0.50     13000
weighted avg       0.70      0.60      0.64     13000



In [35]:
# -> use SMOTE = 0.9

# Submission file - DTC

x_test_final = test
final_prediction = dt3.predict(x_test_final)

submission_1 = pd.DataFrame({"QuoteNumber": test["QuoteNumber"],"QuoteConversion_Flag": final_prediction})
submission_1.to_csv('/Users/jiwonmoon/Desktop/CIS 508/Data Mining I/Assignments/Individual Assignment 3/submissions/submission_DTC_balanced.csv',
                    header=True, index=False)

In [36]:
# Submission file - RFC

x_test_final = test
final_prediction = rfc3.predict(x_test_final)

submission_2 = pd.DataFrame({"QuoteNumber": test["QuoteNumber"],"QuoteConversion_Flag": final_prediction})
submission_2.to_csv('/Users/jiwonmoon/Desktop/CIS 508/Data Mining I/Assignments/Individual Assignment 3/submissions/submission_RFC_balanced.csv',
                    header=True, index=False)

In [37]:
# Submission file - LinearSVM

x_test_final = test
final_prediction = linsvm3.predict(x_test_final)

submission_3 = pd.DataFrame({"QuoteNumber": test["QuoteNumber"],"QuoteConversion_Flag": final_prediction})
submission_3.to_csv('/Users/jiwonmoon/Desktop/CIS 508/Data Mining I/Assignments/Individual Assignment 3/submissions/submission_SVM_balanced.csv',
                    header=True, index=False)

In [38]:
# Submission file - MLPClassifier

x_test_final = test
final_prediction = mlp3.predict(x_test_final)

submission_4 = pd.DataFrame({"QuoteNumber": test["QuoteNumber"],"QuoteConversion_Flag": final_prediction})
submission_4.to_csv('/Users/jiwonmoon/Desktop/CIS 508/Data Mining I/Assignments/Individual Assignment 3/submissions/submission_MLP_balanced.csv',
                    header=True, index=False)

In [39]:
# Submission file - KNN

x_test_final = test
final_prediction = knn3.predict(x_test_final)

submission_5 = pd.DataFrame({"QuoteNumber": test["QuoteNumber"],"QuoteConversion_Flag": final_prediction})
submission_5.to_csv('/Users/jiwonmoon/Desktop/CIS 508/Data Mining I/Assignments/Individual Assignment 3/submissions/submission_KNN_balanced.csv',
                    header=True, index=False)

# Ensemble Stacking

In [40]:
pip install vecstack

Note: you may need to restart the kernel to use updated packages.


In [41]:
from vecstack import stacking

print("___________________________________________________________________________________________\nEnsemble Methods Predictions\n")

models = [ KNeighborsClassifier(), MLPClassifier(), LinearSVC(), RandomForestClassifier(), DecisionTreeClassifier() ]
      
S_train, S_test = stacking(models,                   
                           X_res3, y_res3, test,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

___________________________________________________________________________________________
Ensemble Methods Predictions

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [5]

model  0:     [KNeighborsClassifier]
    fold  0:  [0.71071785]
    fold  1:  [0.71904287]
    fold  2:  [0.71489107]
    fold  3:  [0.71185004]
    ----
    MEAN:     [0.71412546] + [0.00322321]
    FULL:     [0.71412548]

model  1:     [MLPClassifier]
    fold  0:  [0.76161515]
    fold  1:  [0.85822532]
    fold  2:  [0.91220898]
    fold  3:  [0.87247620]
    ----
    MEAN:     [0.85113141] + [0.05533885]
    FULL:     [0.85113039]

model  2:     [LinearSVC]
    fold  0:  [0.67522433]
    fold  1:  [0.59950150]
    fold  2:  [0.56074580]
    fold  3:  [0.55979859]
    ----
    MEAN:     [0.59881755] + [0.04693187]
    FULL:     [0.59881851]

model  3:     [RandomForestClassifier]
    fold  0:  [0.93713858]
    fold  1:  [0.93958126]
   

# Hyperparameter Tuning

In [42]:
#Hyperparameter tuning for RandomForestClassifier

parameters = {'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}

rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=15)
rfc_random.fit(S_train, y_res3)
grid_parm_rfc = rfc_random.best_params_
print(grid_parm_rfc)

{'min_samples_split': 50, 'max_depth': 1}


In [43]:
#Construct Random Forest with best parameters

rfc4 = RandomForestClassifier(**grid_parm_rfc)
rfc4.fit(S_train,y_res3)
rfc4_predict = rfc4.predict(S_test)

In [44]:
# Submission file - StackTuned

rfc4 = pd.DataFrame({"QuoteNumber": test["QuoteNumber"],"QuoteConversion_Flag": rfc4_predict})
rfc4.to_csv('/Users/jiwonmoon/Desktop/CIS 508/Data Mining I/Assignments/Individual Assignment 3/submissions/submission_StackTuned.csv',
                    header=True, index=False)